## Import Libraries

In [1]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pyperclip

import pandas as pd
import time

### Clipboard copy

In [2]:
def clipboard_input(driver, user_input):
    # save existing clipboard data
    existing_clipboard = pyperclip.paste()

    # copy user_input to clipboard. then paste it
    pyperclip.copy(user_input)
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()

    # recover the original clipboard data
    pyperclip.copy(existing_clipboard)  
    time.sleep(1)

### Load user info

In [3]:
def load_user_info():
    f = open('user_info.txt', 'r')
    user_info = f.readlines()
    f.close()
    
    return user_info[0][:-1], user_info[1][:-1]

### Naver Login

In [4]:
def login_naver(driver, user_id, user_pw):
    # naver login page
    driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/aside/')

    # input id 
    driver.find_element_by_xpath('//*[@id="id"]').click()
    clipboard_input(driver, user_id)

    # input pw
    driver.find_element_by_xpath('//*[@id="pw"]').click()
    clipboard_input(driver, user_pw)

    # click login btn
    driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
    time.sleep(1)

    # click 
    driver.find_element_by_xpath('//span[@class="btn_cancel"]').click()

### Enter Details to search in Joonggonara

In [5]:
def get_idx_of_selected_option(option_list, input_text):
    for idx, option in enumerate(option_list):
        if option.text == input_text:
            return idx
    return -1

In [6]:
def enter_details(driver, search_option, item, detail_option):    
    # show 50 items
    driver.find_element_by_xpath('//div[@id="listSizeSelectDiv"]').click()
    time.sleep(1)
    listSize_list = driver.find_elements_by_xpath('//div[@id="listSizeSelectDiv"]/ul/li')
    listSize_list[-1].click()
    time.sleep(1)
    
    # set search options
    driver.find_element_by_xpath('//div[@id="searchOptionSelectDiv"]').click()
    searchBoard_list = driver.find_elements_by_xpath('//div[@id="searchOptionSelectDiv"]/ul/li')
    searchBoard_list[get_idx_of_selected_option(searchBoard_list, search_option['board'])].click()
    time.sleep(1)
    
    driver.find_element_by_xpath('//div[@id="divSearchDateTop"]').click()
    searchDate_list = driver.find_elements_by_xpath('//div[@id="divSearchDateTop"]/ul/li')
    searchDate_list[get_idx_of_selected_option(searchDate_list, search_option['period'])].click()
    
    driver.find_element_by_xpath('//div[@id="divSearchMenuTop"]').click()
    searchMenu_list = driver.find_elements_by_xpath('//div[@id="divSearchMenuTop"]/ul/li')
    searchMenu_list[get_idx_of_selected_option(searchMenu_list, search_option['menu'])].click()
    
    driver.find_element_by_xpath('//div[@id="divSearchByTop"]').click()
    searchBy_list = driver.find_elements_by_xpath('//div[@id="divSearchByTop"]/ul/li')
    searchBy_list[get_idx_of_selected_option(searchBy_list, search_option['by'])].click()
    
    # enter item
    driver.find_element_by_xpath('//input[@placeholder="검색어를 입력해주세요"]').send_keys(item)
    
    # click detail search btn
    driver.find_element_by_xpath('//*[@id="detailSearchBtn"]').click()

    # enter detail options
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 모두 포함"]').send_keys(detail_option['keywords_and'])
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 제외"]').send_keys(detail_option['keywords_not'])
    driver.find_element_by_xpath('//input[@placeholder="다음 단어 중 1개 이상 포함"]').send_keys(detail_option['keywords_or'])
    driver.find_element_by_xpath('//input[@placeholder="다음 어절, 어구 정확히 일치"]').send_keys(detail_option['sentence'])
    
    
    # search
    driver.find_element_by_xpath('//form[@name="frmSearchTop"]/div[@class="input_search_area"]/button[@class="btn-search-green"]').click()
    
    time.sleep(1)

### Get Product Info

In [7]:
def get_post_info(driver, href):
    driver.get(href)
    time.sleep(1)
    driver.switch_to.frame('cafe_main')
    soup = bs(driver.page_source, 'html.parser')

    # post_title
    title = soup.select('div.tit-box span.b')[0].get_text()
    
    # posted_cost
    try:
        cost = soup.select('span.cost')[0].get_text()
    except:
        cost = 0

    # merge contents to single text
    content_tags = soup.select('#tbody')[0].select('p')
    content = ' '.join([ tags.get_text() for tags in content_tags ])
    
    time.sleep(1)

    return {'cost':cost, 'title' : title, 'content' : content}

#### Run ChromeDriver

In [8]:
driver = webdriver.Chrome()
driver.implicitly_wait(2)

#### Login Naver

In [9]:
user_id, user_pw = load_user_info()
login_naver(driver, user_id, user_pw)

#### Search items from Joonggonara and get boards

In [10]:
# log onto Joonggonara and enter details
driver.get('https://cafe.naver.com/joonggonara?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=10050146%26search.searchBy=0')
time.sleep(1)
driver.switch_to.frame(driver.find_element_by_name("cafe_main"))

search_option={
    'board' : "게시글 전체",
    'period' : "전체기간",
    'menu' : "주변기기/악세사리",
    'by' : "제목만"
}

item = "에어팟"

detail_option = {
    'keywords_and' : "미개봉 에어팟 2",
    'keywords_not' : "중고폰 삽니다 프로", 
    'keywords_or' : "", 
    'sentence' : ""
}

num_of_items=500

enter_details(driver, search_option, item, detail_option)

In [11]:
board_navigator = driver.find_element_by_xpath('//div[@class="prev-next"]')
board_list = board_navigator.find_elements_by_tag_name('a')
board_href = board_list[0].get_attribute('href')[:-1]
board_idx = 0

#### Get Posts Links

In [12]:
post_key_list=[]
author_list=set([])
while True:
    # change board
    board_idx+=1
    driver.get(board_href+str(board_idx))
    time.sleep(1)
    driver.switch_to.frame(driver.find_element_by_name("cafe_main"))
    
    # get posts in current board
    posts = driver.find_elements_by_css_selector('div.article-board > table > tbody > tr')

    for post in posts:
        
        # get valid posts
        try:
            author = post.find_element_by_class_name('td_name').text.strip()
            href = post.find_element_by_class_name('article').get_attribute('href')
        except:
            continue
            
        # filter duplicated postings by author
        if author in author_list:
            continue
        post_key_list.append({"author": author, "href":href})
        author_list.add(author)
        
    # check
    print("read {} pages {} posts".format(board_idx, len(post_key_list)))
    
    # terminate condition
    if len(posts) == 0 or len(post_key_list)>num_of_items:
        break

read 1 pages 20 posts
read 2 pages 55 posts
read 3 pages 77 posts
read 4 pages 102 posts
read 5 pages 105 posts
read 6 pages 130 posts
read 7 pages 148 posts
read 8 pages 154 posts
read 9 pages 174 posts
read 10 pages 184 posts
read 11 pages 215 posts
read 12 pages 251 posts
read 13 pages 288 posts
read 14 pages 328 posts
read 15 pages 370 posts
read 16 pages 402 posts
read 17 pages 428 posts
read 18 pages 455 posts
read 19 pages 483 posts
read 20 pages 509 posts


#### Get Price Info

In [13]:
price_info=[]
for post_key in post_key_list:
    try:
        post_info = get_post_info(driver, post_key["href"])
        print(post_info['cost'], post_key["author"], post_info['title'])
        price_info.append(post_info)
    except:
        print("cannot attach post")

price_info_pd=pd.DataFrame(price_info)
price_info_pd.to_csv(item+"_price_info.csv")

160,000원 daaaas 애플정품 에어팟 2세대 무선 미개봉 팝니다
120,000원 무노4025 에어팟2 미개봉 유선상품 판매
140,000원 깅크 에어팟2  유선충전 미개봉 
145,000원 wognekrja 에어팟2 유선 미개봉 새상품팝니다.
143,000원 Care 에어팟 2세대 유선충전 미개봉 판매합니다!
17,000원 햄툐리a 미개봉 미사용 비닐 뜯지 않은 에어팟2 판매 합니다,
150,000원 동원참치 서울)잠실)애플코리아정발 에어팟2세대 유선 한국정품 미개봉
160,000원 ounga2040 에어팟2세대 무선 국내정품 미개봉 -16만-
175,000원 딱딱맨 애플 에어팟 2세대 무선충전 미개봉 새제품 2개 팝니다
168,000원 신속안전1초페이 에어팟 2세대 무선충전 미개봉 정품 판매합니다.
150,000원 빡틀러 에어팟2 유선 충전 미개봉
160,000원 밍대생 ***************에어팟2 무선 미개봉 새상품 팝니다********************
168,000원 중고나라1초머니 에어팟 2세대 무선충전 미개봉 정품 판매합니다.
150,000원 gudfo5585 (천안) 에어팟2 유선 미개봉새상품 판매합니다~~~@@@@@15만원
168,000원 신속안전1초머니 에어팟 2세대 무선충전 미개봉 정품 판매합니다.
168,000원 신속안전1초티켓 에어팟 2세대 무선충전 미개봉 정품 판매합니다.
145,000원 흠냥 충남 아산 에어팟 2세대 유선충전 미개봉
135,000원 아가슈슈 에어팟2 유선 미개봉 13.5만 + 몬카본 라이트닝 케이블
168,000원 신속안전지킴이 에어팟 2세대 무선충전 미개봉 정품 판매합니다.
168,000원 1초머니입니다 에어팟 2세대 무선충전 미개봉 정품 판매합니다.
168,000원 1초머니 에어팟 2세대 무선충전 미개봉 정품 판매합니다.
148,000원 1초머니C 에어팟 2세대 유선 미개봉 정품 판매합니다.
145,000원 dfffffff [평택] 에어팟2 유선충전 미개봉 팔아요
150,000원 모든다2 에어팟2 

155,000원 elly0410 에어팟 2세대 유선충전 미개봉
150,000원 싱싱77 에어팟 2세대 유선충전 미개봉 새상품+케이스+철가루방지스티커
200,000원 dudgns7250 에어팟2 미개봉상품 구매합니다@@@@@@
155,000원 lejh9742 에어팟2세대 유선 정품 미개봉 제품팝니다. (광주)
152,000원 거래는 쿨하게 에어팟2 유선충전 미개봉
140,000원 근혜혜 에어팟 2세대 유선충전 미개봉 운포 14
145,000원 qwdx2323 에어팟 2세대 에어팟2 미개봉 새상품 팝니다
140,000원 16한화우승 에어팟2세대 유선 미개봉새제품팝니다
170,000원 티우스라다 에어팟 2세대 무선충전 미개봉 새제품 (부천 중동 or 택배)
145,000원 yunsix0704 에어팟 2세대 유선충전 모델 팝니다. (미개봉 새제품)
170,000원 발칵한 용여솨 에어팟2 무선 미개봉 새제품
135,000원 피키안코 에어팟2세대 미개봉판매합니다
160,000원 Harim (미개봉) 에어팟2 무선 팝니다
195,000원 Alice airpods2 에어팟2 정품 미개봉 새상품 (☆가격다운 내용확인필)
130,000원 도라기23 에어팟2 미개봉 인천근방 직거래로 구해봅니다.
185,000원 니냐니뇨 에어팟2 무선충전 미개봉 새제품
190,000원 포항사는사람 에어팟2(무선충전) 미개봉 19만원 택배비부담해드립니다
165,000원 pjss 에어팟 2 무선 미개봉 새상품
15,000원 마미맘스 대구 에어팟2 유선(애플코리아 정품10월 미개봉)
150,000원 erjm12 [애플 코리아 정품] 에어팟 2세대 유선 미개봉 새제품 판매합니다.
160,000원 파파유나 [대구] 애플 에어팟2(유선) 미개봉 상품 판매
180,000원 장주황 에어팟2 무선충전용 미개봉제품
150,000원 cdoa2006 에어팟2 유선충전 미개봉 팝니다 부천직거래
145,000원 망고릭 미개봉 정품 에어팟 2세대 유선충전 판매합니다
175,000원 lch2000i 에어팟 2세대 무선충전 모

130,000원 뽀삐언니 에어팟2 유선충전 미개봉 판매합니다~
140,000원 큰동 에어팟2 유선충전 미개봉 새제품팝니다(부산직거래)
150,000원 스라잇 에어팟 2세대 국내정품 유선충전 미개봉 새제품 판매합니다.
160,000원 레토르트 에어팟 2세대 유선충전모델 + 케이스 (미개봉)
145,000원 sunmint3 에어팟 2세대 유선 미개봉 팝니다아
160,000원 거래는깰끔하게 에어팟2 무선충전 미개봉 판매 16만원
165,000원 Christina 에어팟 2세대 미개봉 16.5만 완료
160,000원 BIGJJ123 에어팟2세대 무선 미개봉
170,000원 BeyondZ 에어팟 2세대 무선충전 모델 (미개봉) 서울직거래
140,000원 247 에어팟2 유선 미개봉+실케+정전기스티커
160,000원 sunazzangk 에어팟2세대 유선 (미개봉)
150,000원 wywy224 에어팟2세대 유선충전, 미개봉, 새상품
145,000원 신사입니단 에어팟2세대 (미개봉)
150,000원 imcoro 15미개봉 에어팟2(유선) 판매합니다. 깔끔,정확한 거래원해요~! 택배비 포함
150,000원 lima1106 에어팟2세대 유선 미개봉 15만원에 팔아요
150,000원 어둠의 경로 [구미]미개봉 에어팟2 유선 판매합니다.
130,000원 으오오오옹 부산 에어팟 2세대 유선  미개봉 13만원 구매합니다 
150,000원 너매 에어팟2 유선충전 미개봉
150,000원 래가용 에어팟2세대 유선 미개봉상품 15만 팝니다
145,000원 kanggu0213 에어팟2 유선 미개봉 10월 생산모델 팝니다.
145,000원 bonsoirfleur [미개봉새상품] 애플 에어팟2세대 유선충전 새상품
15,500원 북극극곰 애플 에어팟 2세대 유선충전 미개봉 새상품
160,000원 6856781757738814 에어팟2 무선 미개봉(16만)
135,000원 재미이 에어팟2 유선 미개봉판매합니다
170,000원 레이림 [미개봉]애플코리아 정품 에어팟 2세대 무선 팝니다.
150,000